In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import scipy
import IPython.display

import librosa
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from collections import Counter

# load other modules --> repo root path
sys.path.insert(0, "../")

from utils import text
from utils import audio
from utils.logging import Logger
from params.params import Params as hp
from dataset.dataset import TextToSpeechDataset, TextToSpeechDatasetCollection

In [ ]:
hp.sample_rate = 22050
hp.stft_window_ms = 50
hp.stft_shift_ms = 12.5
hp.num_fft = 1102
hp.num_mels = 80
hp.use_preemphasis = True

#waveform = audio.load("../data/vctk/wav48/p226/p226_012.wav")
waveform = audio.load("../data/ljspeech/wavs/LJ002-0001.wav")
print(audio.duration(waveform))

melspec = audio.mel_spectrogram(waveform)
spec = audio.spectrogram(waveform)

Logger._plot_spectrogram(melspec);
Logger._plot_spectrogram(spec);

print(spec.shape)
print(melspec.shape)

In [ ]:
IPython.display.Audio(data=waveform, rate=hp.sample_rate)

In [ ]:
hp.griffin_lim_iters = 60

inverse_melspec = audio.inverse_mel_spectrogram(melspec)
IPython.display.Audio(data=inverse_melspec, rate=hp.sample_rate)

In [ ]:
hp.griffin_lim_iters = 60

inverse_melspec = audio.inverse_spectrogram(spec)
IPython.display.Audio(data=inverse_melspec, rate=hp.sample_rate)

In [ ]:
hp.griffin_lim_iters = 60

y = waveform
if hp.use_preemphasis: y = scipy.signal.lfilter([1, -hp.preemphasis], [1], y)
wf = int(hp.sample_rate * hp.stft_window_ms / 1000)
hf = int(hp.sample_rate * hp.stft_shift_ms / 1000)
S = librosa.stft(y, n_fft=hp.num_fft , hop_length=hf, win_length=wf)
y = librosa.istft(S, hop_length=hf, win_length=wf)
if hp.use_preemphasis: y = scipy.signal.lfilter([1], [1, -hp.preemphasis], y)

IPython.display.Audio(data=y, rate=hp.sample_rate)

In [ ]:
hp.griffin_lim_iters = 60

SS = librosa.amplitude_to_db(np.abs(S), top_db=None)
inverse_melspec = audio.inverse_spectrogram(SS)
IPython.display.Audio(data=inverse_melspec, rate=hp.sample_rate)

In [ ]:
Logger._plot_spectrogram(np.angle(S))